In [2]:
import os
import chromadb
from chromadb.utils import embedding_functions
from pytube import YouTube
import whisper
import os
import tempfile
import time
from langchain.text_splitter import RecursiveCharacterTextSplitter
import uuid
from openai import OpenAI

In [3]:
chroma_client = chromadb.Client()

sentence_transformer_ef = embedding_functions.SentenceTransformerEmbeddingFunction(model_name='all-MiniLM-L6-v2')
metadata_options = {"hnsw:space": "cosine" }

collection = chroma_client.get_or_create_collection(name="video-transcription", metadata=metadata_options, embedding_function=sentence_transformer_ef)

/opt/miniconda3/envs/youtube-chat/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [4]:
# Every programming language explained in 15 minutes | Prime Reacts - ThePrimeTime
# YOUTUBE_VIDEO = "https://www.youtube.com/watch?v=jQiYW3RXkFU" # Change video URL here
YOUTUBE_VIDEO = "https://www.youtube.com/watch?v=wR2WIRv2zZk" # Change video URL here

In [5]:
# transcribe a youtube video
if not os.path.exists("transcription.txt"):
    youtube = YouTube(YOUTUBE_VIDEO)
    audio = youtube.streams.filter(only_audio=True).first()
    whisper_model = whisper.load_model("base") #whisper.available_models()

    with tempfile.TemporaryDirectory() as tmpdir:
        file = audio.download(output_path=tmpdir)
        start_time = time.time()
        transcription = whisper_model.transcribe(file, fp16=False)["text"].strip()
        end_time = time.time()
        with open("transcription.txt", "w") as file:
            file.write(transcription)
        transcription_time = end_time - start_time
        print(f"Transcription took {round(transcription_time/ 60, 3)} minutes.")
else:
    with open("transcription.txt") as file:
        transcription = file.read()
        
text_splitter = RecursiveCharacterTextSplitter(chunk_size=400, chunk_overlap=20)
documents = text_splitter.split_text(transcription)

In [6]:
print(documents[0])

Every language ever assembly a low-level programming language allowing here Wow, we went did this thing does not This is dead serious. Okay, 15 minutes every single language every single language ever I cannot wait to hear about everybody's favorite language synergy Okay, we've got to start with assembly it makes sense It's to give instructions to the computer's hardware you see computers can only


In [7]:
#upsert documents into ChromaDB
for doc in documents:
    uuid_name = uuid.uuid1()
    # print("document for", uuid_name)
    collection.add(ids=[str(uuid_name)], documents=doc)

In [8]:
print(collection.name, end='\n\n')
print(collection.metadata, end='\n\n')
print(collection.peek().keys(), end='\n\n')
print(collection.peek()['documents'], end='\n\n')
print(collection._embedding_function.models, end='\n\n')

video-transcription

{'hnsw:space': 'cosine'}

dict_keys(['ids', 'embeddings', 'metadatas', 'documents', 'uris', 'data', 'included'])

["Every language ever assembly a low-level programming language allowing here Wow, we went did this thing does not This is dead serious. Okay, 15 minutes every single language every single language ever I cannot wait to hear about everybody's favorite language synergy Okay, we've got to start with assembly it makes sense It's to give instructions to the computer's hardware you see computers can only", "computers can only do things in terms of zero or one So assembly was created in order to make computer instructions more workable for human beings This works by using an assembler that translates the human readable code into ones and zeros Assembly is one of the world's first pro almost a canisa's keyboard It's almost a canisa's keyboard back in the 60s this thing time traveled This thing has", 'This thing has time traveled look at this look at this guy t

In [9]:
def retrieve_context(query_text):
    results = collection.query(query_texts=[query_text],n_results=6)
    return results['documents']

client = OpenAI(
    base_url = 'http://localhost:11434/v1',
    api_key='ollama', 
)

In [13]:
def chatbot():
  # Create a list to store all the messages for context
  messages = []
  messages.append({"role": "system", "content": "You are a helpful assistant who's been given a transcript of a youtube video. When the user asks a question, pretend you're the video who is taking to them. If the message is a greeting, just respond normally." })  
  
  while True:
    # list to store all the retrievals from Chroma
    context = []
    
    # Prompt user for input
    message = input("User: ")
    print(f"User: {message}"+'\n')
    
    # Exit program if user inputs "quit"
    if message.lower() == "quit":
      break
    
    #retrieve necessary context
    context = retrieve_context(message)
    # context_string = ', '.join([str(elem) for elem in context])
    
    message_with_context = f''' 
    
    You are a helpful bot who answers concisely. Use this context: 
    {context} 
    
    To answer the question: {message}. 
    
    If you don't know the answer from the context, say you don't know.
    '''
    
    # Add each new message to the list
    messages.append({"role": "user", "content": message_with_context})
    
    response = client.chat.completions.create(
      model="llama3",
      messages=[{"role": "user", "content": message_with_context}]
    )
    
    # Print the response and add it to the messages list
    chat_message = response.choices[0].message.content
    print(f"Bot: {chat_message}"+'\n')
    messages.append({"role": "assistant", "content": chat_message})

In [14]:
chatbot()

User: what's the gist of  the video

Bot: Based on the provided context, it seems that the video is about programming languages and related topics. The speaker discusses various programming languages, including Perlin, Wolfram, Pascal, and others. They also mention Full Stack websites and their combination of front-end (client-side) and back-end (server-side) components.

The tone of the video appears to be informal and conversational, with the speaker sharing their enthusiasm for programming and language learning. The video may have featured a whiteboard presentation or similar visual aids to illustrate the concepts being discussed.

Overall, the gist of the video seems to be an introduction to different programming languages and their uses, presented in an engaging and accessible manner.

User: what are the main things an ML engineer should know

Bot: Based on the provided context, it seems that Python is one of the most popular programming languages for creating AI and ML applicatio